##モジュール1

In [21]:
import sqlite3

# ★ 環境構築・接続 ★
# メモリ上のデータベース（実際のファイルとして保存する場合は、ファイル名を指定します）
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# ★ テーブルの作成 ★
# ユーザー情報を格納するテーブル
#users テーブル：ユーザーの id、name、age を格納します。
cur.execute('''
CREATE TABLE users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER
)
''')

# 注文情報を格納するテーブル（users テーブルと外部キーで結合）
#orders テーブル：注文の order_id、user_id（外部キー）、product、price を格納し、users テーブルと結合できるようにしています。
cur.execute('''
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    product TEXT,
    price REAL,
    FOREIGN KEY(user_id) REFERENCES users(id)
)
''')

conn.commit()

# ★ INSERT：データの新規登録 ★
# users テーブルにデータを登録
cur.execute("INSERT INTO users (name, age) VALUES (?, ?)", ("Alice", 30))
cur.execute("INSERT INTO users (name, age) VALUES (?, ?)", ("Bob", 25))
conn.commit()

# orders テーブルにデータを登録
cur.execute("INSERT INTO orders (user_id, product, price) VALUES (?, ?, ?)", (1, "Laptop", 1200.00))
cur.execute("INSERT INTO orders (user_id, product, price) VALUES (?, ?, ?)", (2, "Smartphone", 800.00))
cur.execute("INSERT INTO orders (user_id, product, price) VALUES (?, ?, ?)", (1, "Tablet", 300.00))
conn.commit()

# ★ SELECT：データの取得 ★
print("=== SELECT: 全ユーザーの取得 ===")
cur.execute("SELECT * FROM users")
for row in cur.fetchall():
    print(row)

# ★ UPDATE：データの更新 ★
# 例：Alice の年齢を 31 に更新
cur.execute("UPDATE users SET age = ? WHERE name = ?", (31, "Alice"))
conn.commit()

print("\n=== UPDATE 後のユーザー情報 ===")
cur.execute("SELECT * FROM users")
for row in cur.fetchall():
    print(row)

# ★ DELETE：データの削除 ★
# 例：Bob の情報を削除
cur.execute("DELETE FROM users WHERE name = ?", ("Bob",))
conn.commit()

print("\n=== DELETE 後のユーザー情報 ===")
cur.execute("SELECT * FROM users")
for row in cur.fetchall():
    print(row)

# ★ WHERE：条件抽出 ★
# orders テーブルから、price（価格）が 500 より大きい注文を抽出
print("\n=== WHERE: 価格が 500 より大きい注文 ===")
cur.execute("SELECT * FROM orders WHERE price > ?", (500,))
for row in cur.fetchall():
    print(row)

# ★ GROUP BY：集計 ★
# 各ユーザーごとの注文数を集計
print("\n=== GROUP BY: ユーザーごとの注文数の集計 ===")
cur.execute("SELECT user_id, COUNT(*) as order_count FROM orders GROUP BY user_id")
for row in cur.fetchall():
    print(row)

# ★ JOIN：テーブルの結合 ★
# users テーブルと orders テーブルを user_id をキーに内部結合し、ユーザー名と注文情報を取得
print("\n=== JOIN: ユーザーと注文の結合結果 ===")
cur.execute('''
SELECT users.name, orders.product, orders.price
FROM orders
JOIN users ON orders.user_id = users.id
''')
for row in cur.fetchall():
    print(row)

# 接続をクローズ
conn.close()


=== SELECT: 全ユーザーの取得 ===
(1, 'Alice', 30)
(2, 'Bob', 25)

=== UPDATE 後のユーザー情報 ===
(1, 'Alice', 31)
(2, 'Bob', 25)

=== DELETE 後のユーザー情報 ===
(1, 'Alice', 31)

=== WHERE: 価格が 500 より大きい注文 ===
(1, 1, 'Laptop', 1200.0)
(2, 2, 'Smartphone', 800.0)

=== GROUP BY: ユーザーごとの注文数の集計 ===
(1, 2)
(2, 1)

=== JOIN: ユーザーと注文の結合結果 ===
('Alice', 'Laptop', 1200.0)
('Alice', 'Tablet', 300.0)


In [25]:
import sqlite3

# ★ 環境構築・接続 ★
conn = sqlite3.connect(':memory:')  # メモリ上に一時データベースを作成
cur = conn.cursor()

# ★ テーブルの作成 ★
cur.execute('''
CREATE TABLE users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER
)
''')

cur.execute('''
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    product TEXT,
    price REAL,
    FOREIGN KEY(user_id) REFERENCES users(id)
)
''')
conn.commit()

# ★ INSERT：データの新規登録 ★
# users テーブルに複数のユーザーを登録
cur.execute("INSERT INTO users (name, age) VALUES (?, ?)", ("Alice", 30))
cur.execute("INSERT INTO users (name, age) VALUES (?, ?)", ("Bob", 25))
cur.execute("INSERT INTO users (name, age) VALUES (?, ?)", ("Charlie", 40))
conn.commit()

# orders テーブルに複数の注文情報を登録
cur.execute("INSERT INTO orders (user_id, product, price) VALUES (?, ?, ?)", (1, "Laptop", 1200.00))
cur.execute("INSERT INTO orders (user_id, product, price) VALUES (?, ?, ?)", (2, "Smartphone", 800.00))
cur.execute("INSERT INTO orders (user_id, product, price) VALUES (?, ?, ?)", (1, "Tablet", 300.00))
cur.execute("INSERT INTO orders (user_id, product, price) VALUES (?, ?, ?)", (3, "Laptop", 1200.00))
conn.commit()

# ★ COUNT: 注文数を集計する例 ★
cur.execute("SELECT COUNT(*) FROM orders")
order_count = cur.fetchone()[0]
print("=== COUNT: 注文の総数 ===")
print("注文数:", order_count)

# ★ DISTINCT: 注文テーブルから重複しない商品名を取得する例 ★
cur.execute("SELECT DISTINCT product FROM orders")
distinct_products = cur.fetchall()
print("\n=== DISTINCT: 注文されたユニークな商品名 ===")
for product in distinct_products:
    print(product[0])

# ★ LIMIT: 注文テーブルから最初の 2 件だけ取得する例 ★
cur.execute("SELECT * FROM orders LIMIT 2")
limited_orders = cur.fetchall()
print("\n=== LIMIT: 最初の 2 件の注文 ===")
for order in limited_orders:
    print(order)

# ★ UPDATE: ユーザー情報の更新例 ★
# 例：Alice の年齢を 35 に更新
cur.execute("UPDATE users SET age = ? WHERE name = ?", (35, "Alice"))
conn.commit()

# 更新結果を確認するため、users テーブルの内容を取得
cur.execute("SELECT * FROM users")
updated_users = cur.fetchall()
print("\n=== UPDATE: 更新後のユーザー情報 ===")
for user in updated_users:
    print(user)

# 接続をクローズ
conn.close()


=== COUNT: 注文の総数 ===
注文数: 4

=== DISTINCT: 注文されたユニークな商品名 ===
Laptop
Smartphone
Tablet

=== LIMIT: 最初の 2 件の注文 ===
(1, 1, 'Laptop', 1200.0)
(2, 2, 'Smartphone', 800.0)

=== UPDATE: 更新後のユーザー情報 ===
(1, 'Alice', 35)
(2, 'Bob', 25)
(3, 'Charlie', 40)


In [23]:
!pip install pandas
import sqlite3
import pandas as pd

# データベースに接続（ここでは前と同じメモリ内データベースを再利用する例）
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# ★ テーブルの作成 ★
cur.execute('''
CREATE TABLE users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER
)
''')

cur.execute('''
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    product TEXT,
    price REAL,
    FOREIGN KEY(user_id) REFERENCES users(id)
)
''')
conn.commit()

# ★ INSERT：データの新規登録 ★
cur.execute("INSERT INTO users (name, age) VALUES (?, ?)", ("Alice", 30))
cur.execute("INSERT INTO users (name, age) VALUES (?, ?)", ("Bob", 25))
conn.commit()

cur.execute("INSERT INTO orders (user_id, product, price) VALUES (?, ?, ?)", (1, "Laptop", 1200.00))
cur.execute("INSERT INTO orders (user_id, product, price) VALUES (?, ?, ?)", (2, "Smartphone", 800.00))
cur.execute("INSERT INTO orders (user_id, product, price) VALUES (?, ?, ?)", (1, "Tablet", 300.00))
conn.commit()

# ★ pandas で SQL の結果を DataFrame として読み込む ★

# users テーブルの内容を読み込み
df_users = pd.read_sql_query("SELECT * FROM users", conn)
print("=== users テーブル ===")
print(df_users)

# orders テーブルの内容を読み込み
df_orders = pd.read_sql_query("SELECT * FROM orders", conn)
print("\n=== orders テーブル ===")
print(df_orders)

# JOIN の例も DataFrame にして出力する
query = '''
SELECT users.name, orders.product, orders.price
FROM orders
JOIN users ON orders.user_id = users.id
'''
df_join = pd.read_sql_query(query, conn)
print("\n=== JOIN の結果 ===")
print(df_join)

# 接続をクローズ
conn.close()


=== users テーブル ===
   id   name  age
0   1  Alice   30
1   2    Bob   25

=== orders テーブル ===
   order_id  user_id     product   price
0         1        1      Laptop  1200.0
1         2        2  Smartphone   800.0
2         3        1      Tablet   300.0

=== JOIN の結果 ===
    name     product   price
0  Alice      Laptop  1200.0
1    Bob  Smartphone   800.0
2  Alice      Tablet   300.0


In [24]:
!pip install tabulate

import sqlite3
from tabulate import tabulate

# データベースに接続（ここでは再度例としてメモリ上データベースを使用）
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# テーブル作成とデータ登録（前述の例と同じ処理）
cur.execute('''
CREATE TABLE users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER
)
''')
cur.execute('''
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    product TEXT,
    price REAL,
    FOREIGN KEY(user_id) REFERENCES users(id)
)
''')
conn.commit()

cur.execute("INSERT INTO users (name, age) VALUES (?, ?)", ("Alice", 30))
cur.execute("INSERT INTO users (name, age) VALUES (?, ?)", ("Bob", 25))
conn.commit()

cur.execute("INSERT INTO orders (user_id, product, price) VALUES (?, ?, ?)", (1, "Laptop", 1200.00))
cur.execute("INSERT INTO orders (user_id, product, price) VALUES (?, ?, ?)", (2, "Smartphone", 800.00))
cur.execute("INSERT INTO orders (user_id, product, price) VALUES (?, ?, ?)", (1, "Tablet", 300.00))
conn.commit()

# SELECT 文で取得した結果をリストとして受け取る
cur.execute("SELECT * FROM users")
users_data = cur.fetchall()
# カラム名の取得（sqlite3.Cursor オブジェクトの description 属性から）
users_columns = [desc[0] for desc in cur.description]

print("=== users テーブル (tabulate) ===")
print(tabulate(users_data, headers=users_columns, tablefmt="psql"))

# 接続をクローズ
conn.close()


=== users テーブル (tabulate) ===
+------+--------+-------+
|   id | name   |   age |
|------+--------+-------|
|    1 | Alice  |    30 |
|    2 | Bob    |    25 |
+------+--------+-------+


## モジュール2


以下は、Python で DDL（データ定義言語）の各種操作を実行するサンプルコード例になります。
以下の例では、

・CREATE TABLE でテーブルを作成<br>
・ALTER TABLE でテーブル定義の変更（列の追加など）<br>
・DROP TABLE でテーブルを削除<br>
・TRUNCATE TABLE（または SQLite では同等の処理）でテーブル内データを全削除<br>
・MySQL 用の ALTER および TRUNCATE の例<br>
・SQL スクリプトファイルを用いたテーブル作成とデータロード<br>
といった内容をそれぞれ示しています。

In [1]:
import sqlite3

# SQLite のメモリ上データベースに接続
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# --- CREATE TABLE ---
cur.execute("""
CREATE TABLE employees (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    department TEXT
)
""")
print("【SQLite】テーブル 'employees' を作成しました。")

# --- ALTER TABLE ---
# SQLite の ALTER TABLE は機能が限定的ですが、列の追加は可能です。
cur.execute("ALTER TABLE employees ADD COLUMN salary REAL")
print("【SQLite】テーブル 'employees' に 'salary' 列を追加しました。")

# --- TRUNCATE TABLE の代替 ---
# SQLite は TRUNCATE TABLE をサポートしていないため、DELETE 文で全件削除します。
cur.execute("DELETE FROM employees")
print("【SQLite】テーブル 'employees' のデータを全削除しました。（TRUNCATE の代替）")

# --- DROP TABLE ---
cur.execute("DROP TABLE employees")
print("【SQLite】テーブル 'employees' を削除しました。")

conn.commit()
conn.close()


【SQLite】テーブル 'employees' を作成しました。
【SQLite】テーブル 'employees' に 'salary' 列を追加しました。
【SQLite】テーブル 'employees' のデータを全削除しました。（TRUNCATE の代替）
【SQLite】テーブル 'employees' を削除しました。
